In [5]:

import pandas as pd
import numpy as np
from statsforecast import StatsForecast
from statsforecast.models import AutoETS
from tqdm import tqdm


In [ ]:

def mase(y, y_pred, y_train, seasonality=1):
    """
    Calculate Mean Absolute Scaled Error (MASE)
    y: Actual values
    y_pred: Predicted values
    y_train: Training data for scaling factor
    seasonality: Seasonal period for naive forecasting
    """
    mae = np.mean(np.abs(y - y_pred))
    naive_forecast_errors = np.abs(y_train[seasonality:] - y_train[:-seasonality])
    scaling_factor = np.mean(naive_forecast_errors)
    return mae / scaling_factor


In [6]:

def bottom_up_forecasting(data, m, h, seasonality):
    """
    Perform bottom-up forecasting at the store level by summing up department-level forecasts.

    data: Input dataset with store-department level time series.
    m: Number of data points from the end of the dataset for cross-validation.
    h: Number of steps ahead for forecasting.
    seasonality: Seasonal period for naive forecasting.
    """
    store_ids = data['store_id'].unique()
    results = []

    for store_id in tqdm(store_ids, desc="Processing all stores"):
        # Subset data for the current store
        store_data = data[data['store_id'] == store_id]
        department_ids = store_data['store_dept_id'].unique()

        # Initialize forecasts for aggregation
        autoets_forecasts = []
        naive_forecasts = []

        for dept_id in department_ids:
            # Subset data for the current department
            dept_data = store_data[store_data['store_dept_id'] == dept_id]
            dept_data = dept_data.rename(columns={'d': 'ds', 'revenue': 'y'})

            # Leave-one-out cross-validation
            model = AutoETS(model=["Z", "Z", "Z"], alias="AutoETS", damped=True, season_length=seasonality)
            errors_autoets = []
            errors_naive = []

            for i in range(len(dept_data) - m, len(dept_data) - h + 1):
                train_subset = dept_data.iloc[:i]
                test_subset = dept_data.iloc[i:i + h]

                # AutoETS forecast
                autoets_model = model.fit(train_subset['y'].values)
                y_hat_autoets = autoets_model.predict(h=h).get("mean")
                errors_autoets.extend(zip(test_subset['y'].values, y_hat_autoets))

                # Naive forecast
                y_hat_naive = train_subset['y'].iloc[-seasonality:].values.tolist() * h
                y_hat_naive = y_hat_naive[:h]
                errors_naive.extend(zip(test_subset['y'].values, y_hat_naive))

            # Aggregate forecasts for the department
            dept_autoets_forecast = [e[1] for e in errors_autoets]
            dept_naive_forecast = [e[1] for e in errors_naive]

            autoets_forecasts.append(dept_autoets_forecast)
            naive_forecasts.append(dept_naive_forecast)

        # Aggregate to store level
        store_autoets_forecast = np.sum(autoets_forecasts, axis=0)
        store_naive_forecast = np.sum(naive_forecasts, axis=0)

        # Calculate scaling factor at the store level
        store_level_data = store_data.groupby('d')['revenue'].sum()
        naive_errors = np.abs(store_level_data.values[seasonality:] - store_level_data.values[:-seasonality])
        scaling_factor = np.mean(naive_errors)

        # Calculate MASE for store level
        mase_autoets = np.mean(np.abs(store_level_data.values[-m:] - store_autoets_forecast)) / scaling_factor
        mase_naive = np.mean(np.abs(store_level_data.values[-m:] - store_naive_forecast)) / scaling_factor

        results.append({"store_id": store_id, "AutoETS_MASE": mase_autoets, "Naive_MASE": mase_naive})

    # Create and return the summary table
    summary_table = pd.DataFrame(results)
    print("Summary Table of MASE for Each Store:")
    print(summary_table)
    print(f"\nAverage AutoETS MASE across all stores: {summary_table['AutoETS_MASE'].mean()}")
    print(f"Average Naive MASE across all stores: {summary_table['Naive_MASE'].mean()}")

    return summary_table


In [7]:

# Load data
data = pd.read_csv("C:/Data/M5store_department.csv")

# Add 'store_id' from 'store_dept_id'
data['store_id'] = data['store_dept_id'].str.split('_').str[:2].str.join('_')

# Parameters
m = 28  # Last m points for cross-validation
h = 1  # Forecast horizon
seasonality = 7  # Weekly seasonality for daily data

# Run bottom-up forecasting
summary_table = bottom_up_forecasting(data, m, h, seasonality)


Processing all stores: 100%|███████████████████████████████████████████████████████████| 10/10 [10:28<00:00, 62.88s/it]

Summary Table of MASE for Each Store:
  store_id  AutoETS_MASE  Naive_MASE
0     CA_1      0.737865    0.846793
1     CA_2      1.139215    1.369136
2     CA_3      0.645090    0.847744
3     CA_4      0.791399    0.966743
4     TX_1      0.978194    1.436011
5     TX_2      0.778005    0.990767
6     TX_3      0.937347    1.458267
7     WI_1      0.884681    1.263655
8     WI_2      1.034778    1.604636
9     WI_3      0.821734    1.143391

Average AutoETS MASE across all stores: 0.8748308955131895
Average Naive MASE across all stores: 1.1927142467832386
